# Parent Document Retriever

**문서 검색과 문서 분할의 균형 잡기**

문서 검색 과정에서 문서를 적절한 크기의 조각(청크)으로 나누는 것은 다음의 **상충되는 두 가지 중요한 요소를 고려**해야 합니다.

1. 작은 문서를 원하는 경우: 이렇게 하면 문서의 임베딩이 그 의미를 가장 정확하게 반영할 수 있습니다. 문서가 너무 길면 임베딩이 의미를 잃어버릴 수 있습니다.
2. 각 청크의 맥락이 유지되도록 충분히 긴 문서를 원하는 경우입니다.

**`ParentDocumentRetriever`의 역할**

이 두 요구 사항 사이의 균형을 맞추기 위해 `ParentDocumentRetriever`라는 도구가 사용됩니다. 이 도구는 문서를 작은 조각으로 나누고, 이 조각들을 관리합니다. 검색을 진행할 때는, 먼저 이 작은 조각들을 찾아낸 다음, 이 조각들이 속한 원본 문서(또는 더 큰 조각)의 식별자(ID)를 통해 전체적인 맥락을 파악할 수 있습니다.

여기서 '부모 문서'란, 작은 조각이 나누어진 원본 문서를 말합니다. 이는 전체 문서일 수도 있고, 비교적 큰 다른 조각일 수도 있습니다. 이 방식을 통해 문서의 의미를 정확하게 파악하면서도, 전체적인 맥락을 유지할 수 있게 됩니다.

**정리**

- **문서 간의 계층 구조 활용**: `ParentDocumentRetriever`는 문서 검색의 효율성을 높이기 위해 문서 간의 계층 구조를 활용합니다.
- **검색 성능 향상**: 관련성 높은 문서를 빠르게 찾아내며, 주어진 질문에 대한 가장 적합한 답변을 제공하는 문서를 효과적으로 찾아낼 수 있습니다.
  문서를 검색할 때 자주 발생하는 두 가지 상충되는 요구 사항이 있습니다:


In [ ]:
%pip install -qU deeplake

여러 개의 텍스트 파일을 로드하기 위해 `TextLoader` 객체를 생성하고 데이터를 로드합니다.


In [1]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

In [2]:
loaders = [
    # 파일을 로드합니다.
    TextLoader("./data/ai-story.txt"),
    # 파일을 로드합니다.
    TextLoader("./data/appendix-keywords.txt"),
]

docs = []
for loader in loaders:
    # 로더를 사용하여 문서를 로드하고 docs 리스트에 추가합니다.
    docs.extend(
        loader.load()
    )

## 전체 문서 검색

이 모드에서는 전체 문서를 검색하고자 합니다. 따라서 `child_splitter` 만 지정하도록 하겠습니다.

- 나중에는 `parent_splitter` 도 지정하여 결과를 비교해 보겠습니다.


In [3]:
# 자식 분할기를 생성합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=300)

# DB를 생성합니다.
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)

store = InMemoryStore()

# Retriever 를 생성합니다.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

`retriever.add_documents(docs, ids=None)` 함수로 문서목록을 추가합니다.

- `ids` 가 `None` 이면 자동으로 생성됩니다.
- `add_to_docstore=False` 로 설정시 document 를 중복으로 추가하지 않습니다. 단, 중복을 체크하기 위한 `ids` 값이 필수 값으로 요구됩니다.


In [4]:
# 문서를 검색기에 추가합니다. docs는 문서 목록이고, ids는 문서의 고유 식별자 목록입니다.
retriever.add_documents(docs, ids=None, add_to_docstore=True)

이 코드는 두 개의 키를 반환해야 합니다. 그 이유는 우리가 두 개의 문서를 추가했기 때문입니다.


- `store` 객체의 `yield_keys()` 메서드를 호출하여 반환된 키(key) 값들을 리스트로 변환합니다.


In [5]:
# 저장소의 모든 키를 리스트로 반환합니다.
list(store.yield_keys())

['477b9ba2-d39c-4f42-80ce-08c8f0d1b08a',
 '1ea8adb5-5316-465d-8a41-f01b8d123152']

이제 벡터 스토어 검색 기능을 호출해 보겠습니다.

우리가 작은 청크(chunk)들을 저장하고 있기 때문에, 검색 결과로 작은 청크들이 반환되는 것을 확인할 수 있을 것입니다.


`vectorstore` 객체의 `similarity_search` 메서드를 사용하여 유사도 검색을 수행합니다.


In [6]:
# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Word2Vec")

`sub_docs[0].page_content`를 출력합니다.


In [7]:
# sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(sub_docs[0].page_content)

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


이제 전체 retriever에서 검색해 보겠습니다. 이 과정에서는 작은 청크(chunk)들이 위치한 **문서를 반환** 하기 때문에 상대적으로 큰 문서들이 반환될 것입니다.


`retriever` 객체의 `get_relevant_documents()` 메서드를 사용하여 쿼리와 관련된 문서를 검색합니다.


In [8]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.get_relevant_documents("Word2Vec")

검색된 문서(`retrieved_docs[0]`)의 일부 내용을 출력합니다.


In [9]:
# 검색된 문서의 문서의 페이지 내용의 길이를 출력합니다.
print(
    f"문서의 길이: {len(retrieved_docs[0].page_content)}",
    end="\n\n=====================\n\n",
)

# 문서의 일부를 출력합니다.
print(retrieved_docs[0].page_content[2000:2500])

문서의 길이: 5733


 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을


## 더 큰 크기의 청크로부터 검색

이전의 결과처럼 **전체 문서가 너무 커서 있는 그대로 검색하기에는 부적합** 할 수 있습니다.

이런 경우, 실제로 우리가 하고 싶은 것은 먼저 원시 문서를 더 큰 청크로 분할한 다음, 더 작은 청크로 분할하는 것입니다.

그런 다음 작은 청크들을 인덱싱하지만, 검색 시에는 더 큰 청크를 검색합니다 (그러나 여전히 전체 문서는 아닙니다).


- `RecursiveCharacterTextSplitter`를 사용하여 부모 문서와 자식 문서를 생성합니다.
  - 부모 문서는 `chunk_size`가 1200으로 설정되어 있습니다.
  - 자식 문서는 `chunk_size`가 300으로 설정되어 있으며, 부모 문서보다 작은 크기로 생성됩니다.


In [10]:
# 부모 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=900)
# 자식 문서를 생성하는 데 사용되는 텍스트 분할기입니다.
# 부모보다 작은 문서를 생성해야 합니다.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=300)
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소입니다.
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)
# 부모 문서의 저장 계층입니다.
store = InMemoryStore()

`ParentDocumentRetriever`를 초기화하는 코드입니다.

- `vectorstore` 매개변수는 문서 벡터를 저장하는 벡터 저장소를 지정합니다.
- `docstore` 매개변수는 문서 데이터를 저장하는 문서 저장소를 지정합니다.
- `child_splitter` 매개변수는 하위 문서를 분할하는 데 사용되는 문서 분할기를 지정합니다.
- `parent_splitter` 매개변수는 상위 문서를 분할하는 데 사용되는 문서 분할기를 지정합니다.

`ParentDocumentRetriever`는 계층적 문서 구조를 처리하며, 상위 문서와 하위 문서를 별도로 분할하고 저장합니다. 이를 통해 검색 시 상위 문서와 하위 문서를 효과적으로 활용할 수 있습니다.


In [11]:
retriever = ParentDocumentRetriever(
    # 벡터 저장소를 지정합니다.
    vectorstore=vectorstore,
    # 문서 저장소를 지정합니다.
    docstore=store,
    # 하위 문서 분할기를 지정합니다.
    child_splitter=child_splitter,
    # 상위 문서 분할기를 지정합니다.
    parent_splitter=parent_splitter,
)

`retriever` 객체에 `docs`를 추가합니다. `retriever`가 검색할 수 있는 문서 집합에 새로운 문서들을 추가하는 역할을 합니다.


In [12]:
retriever.add_documents(docs)  # 문서를 retriever에 추가합니다.

이제 문서의 수가 훨씬 더 많아진 것을 볼 수 있습니다. 이는 더 큰 청크(chunk)들입니다.


In [13]:
# 저장소에서 키를 생성하고 리스트로 변환한 후 길이를 반환합니다.
len(list(store.yield_keys()))

20

기본 벡터 저장소가 여전히 작은 청크를 검색하는지 확인해 보겠습니다.


`vectorstore` 객체의 `similarity_search` 메서드를 사용하여 유사도 검색을 수행합니다.


In [14]:
# 유사도 검색을 수행합니다.
sub_docs = vectorstore.similarity_search("Word2Vec")
# sub_docs 리스트의 첫 번째 요소의 page_content 속성을 출력합니다.
print(sub_docs[0].page_content)

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)


이번에는 `retriever` 객체의 `get_relevant_documents` 메서드를 사용하여 문서를 검색합니다.


In [15]:
# 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.get_relevant_documents("Word2Vec")

# 검색된 문서의 첫 번째 문서의 페이지 내용의 길이를 반환합니다.
print(retrieved_docs[0].page_content)

정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.
예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.
예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.
연관키워드: 자연어 처리, 딥러닝, 텍스트 생성

FAISS (Facebook AI Similarity Search)


In [17]:
# 검색된 문서 중 첫 번째 문서의 내용을 출력합니다.
print(retrieved_docs[0].page_content)

정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.
예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.
연관키워드: 혁신, 기술, 비즈니스 모델

Crawling

정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.
예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.
연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진

Word2Vec

정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.
연관키워드: 자연어 처리, 임베딩, 의미론적 유사성
LLM (Large Language Model)

정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.
예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.
연관키워드: 자연어 처리, 딥러닝, 텍스트 생성

FAISS (Facebook AI Similarity Search)
